<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/Avaliacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install sklearn-crfsuite
! pip install eli5
! pip install unidecode
! pip install stanza

     |████████████████████████████████| 965 kB 19.2 MB/s 
     |████████████████████████████████| 106 kB 15.8 MB/s 
     |████████████████████████████████| 235 kB 15.5 MB/s 
     |████████████████████████████████| 432 kB 14.7 MB/s 
     |████████████████████████████████| 175 kB 41.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=0764aed4cce1db3f6a09e5a33ef594cb0b357c4964d1904b862601e209ee12dc
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [2]:
import pandas as pd     
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk  
nltk.download('punkt')
from nltk.corpus import stopwords # Import the stop word list

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from time import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Avaliações com o dataset de restaurantes

In [ ]:
data = pd.read_csv('/content/restaurantes.csv',sep=';')

In [ ]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline

In [ ]:
from unidecode import unidecode
import re

words = []

for word in data['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())

In [ ]:
data['text'] = words
data['labels'] =-1
data.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,-1
1,excelente peixe para quem esta localizado no ...,peixe,Food,-1
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks,-1
3,muito bom o atendimento,atendimento,Service,-1
4,cardapio variado,Cardápio,Food,-1


In [ ]:
data.loc[data["aspectsClass"] == "Service", "labels"] = 0
data.loc[data["aspectsClass"] == "Food", "labels"] = 1
data.loc[data["aspectsClass"] == "Price", "labels"] = 2
data.loc[data["aspectsClass"] == "Ambience", "labels"] = 3
data.loc[data["aspectsClass"] == "Location", "labels"] = 4

In [ ]:
df = data[data['labels'] != -1]
df.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,1
1,excelente peixe para quem esta localizado no ...,peixe,Food,1
3,muito bom o atendimento,atendimento,Service,0
4,cardapio variado,Cardápio,Food,1
5,ambiente agradavel,Ambiente,Ambience,3


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["labels"], test_size = 0.3, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency
X_train_tf = tf_vectorizer.fit_transform(X_train)
X_test_tf = tf_vectorizer.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [ ]:
from sklearn import metrics
t = time()
naive_bayes_classifier.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')
#print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

train time: 0.003s
test time:  0.000s
accuracy:   0.492
              precision    recall  f1-score   support

     Service       0.38      0.38      0.38        13
        Food       0.69      0.69      0.69        26
       Price       0.00      0.00      0.00         4
    Ambience       0.47      0.47      0.47        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.49        61
   macro avg       0.31      0.31      0.31        61
weighted avg       0.49      0.49      0.49        61

confusion matrix:
[[ 5  4  1  3  0]
 [ 2 18  1  2  3]
 [ 1  1  0  1  1]
 [ 4  3  1  7  0]
 [ 1  0  0  2  0]]
------------------------------


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dTree = DecisionTreeClassifier()

t = time()
dTree.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = dTree.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.017s
test time:  0.002s
accuracy:   0.541
              precision    recall  f1-score   support

     Service       0.44      0.54      0.48        13
        Food       0.64      0.62      0.63        26
       Price       0.00      0.00      0.00         4
    Ambience       0.67      0.53      0.59        15
    Location       0.40      0.67      0.50         3

    accuracy                           0.54        61
   macro avg       0.43      0.47      0.44        61
weighted avg       0.55      0.54      0.54        61

confusion matrix:
[[ 7  3  2  1  0]
 [ 4 16  1  3  2]
 [ 1  2  0  0  1]
 [ 4  3  0  8  0]
 [ 0  1  0  0  2]]
------------------------------


In [ ]:
from sklearn.svm import SVC
svm = SVC()

t = time()
svm.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = svm.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.020s
test time:  0.002s
accuracy:   0.475
              precision    recall  f1-score   support

     Service       0.67      0.46      0.55        13
        Food       0.48      0.77      0.59        26
       Price       0.00      0.00      0.00         4
    Ambience       0.50      0.20      0.29        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.48        61
   macro avg       0.33      0.29      0.28        61
weighted avg       0.47      0.48      0.44        61

confusion matrix:
[[ 6  5  1  1  0]
 [ 2 20  0  2  2]
 [ 1  2  0  0  1]
 [ 0 12  0  3  0]
 [ 0  3  0  0  0]]
------------------------------


In [ ]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression()

t = time()
regression.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = regression.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.068s
test time:  0.000s
accuracy:   0.574
              precision    recall  f1-score   support

     Service       0.64      0.54      0.58        13
        Food       0.65      0.77      0.70        26
       Price       0.00      0.00      0.00         4
    Ambience       0.57      0.53      0.55        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.57        61
   macro avg       0.37      0.37      0.37        61
weighted avg       0.55      0.57      0.56        61

confusion matrix:
[[ 7  3  1  2  0]
 [ 2 20  0  2  2]
 [ 1  2  0  0  1]
 [ 1  5  1  8  0]
 [ 0  1  0  2  0]]
------------------------------


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

t = time()
mlp.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = mlp.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.675s
test time:  0.002s
accuracy:   0.557
              precision    recall  f1-score   support

     Service       0.50      0.46      0.48        13
        Food       0.71      0.77      0.74        26
       Price       0.00      0.00      0.00         4
    Ambience       0.53      0.53      0.53        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.56        61
   macro avg       0.35      0.35      0.35        61
weighted avg       0.54      0.56      0.55        61

confusion matrix:
[[ 6  4  2  1  0]
 [ 1 20  0  3  2]
 [ 1  1  0  1  1]
 [ 3  3  1  8  0]
 [ 1  0  0  2  0]]
------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
forest = RandomForestClassifier() 

forest = forest.fit(X_train_tf, y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = forest.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 365.734s
test time:  0.026s
accuracy:   0.574
              precision    recall  f1-score   support

     Service       0.80      0.62      0.70        13
        Food       0.61      0.77      0.68        26
       Price       0.00      0.00      0.00         4
    Ambience       0.54      0.47      0.50        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.57        61
   macro avg       0.39      0.37      0.37        61
weighted avg       0.56      0.57      0.56        61

confusion matrix:
[[ 8  2  1  2  0]
 [ 2 20  0  2  2]
 [ 0  2  0  1  1]
 [ 0  7  1  7  0]
 [ 0  2  0  1  0]]
------------------------------


# Avaliações com o dataset de TVs

In [3]:
import json
tvs = json.load(open("/content/tv.json"))

In [4]:
texts = []
aspectsClass = []
for x in tvs['reviews']:
  texts.append(x['review'])
  y = x['implicit aspects']
  if(len(y) > 0):
    aspectsClass.append(y[0])
  else:
    aspectsClass.append(None)

d = {'text': texts, 'aspectsClass':aspectsClass}

df_tvs = pd.DataFrame(data=d)
df_tvs.head()

,text,aspectsClass
0,Excelente smart tv. E foi entregue vem antes d...,entrega
1,"Imagem ótima recursos de web bons , recomendo ...",produto
2,"produto ótimo, entrega antes do prazo previsto...",None
3,"Muito bom o produto, esta tv está entre as das...",funcionalidades
4,a entrega foi super rápido... muito antes do p...,None


In [5]:
values = [None, 'None']
df_tst = df_tvs[df_tvs.aspectsClass.isin(values) == False]
df_tst.head()

,text,aspectsClass
0,Excelente smart tv. E foi entregue vem antes d...,entrega
1,"Imagem ótima recursos de web bons , recomendo ...",produto
3,"Muito bom o produto, esta tv está entre as das...",funcionalidades
11,Todos os aparelhos samsung são de uma qualidad...,produto
12,"Atendeu todas as expectativas, recomendo Samsu...",produto


In [7]:
from unidecode import unidecode
import re

words = []

for word in df_tst['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())


df_tst['text'] = words
df_tst.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,text,aspectsClass
0,excelente smart tv e foi entregue vem antes d...,entrega
1,imagem otima recursos de web bons recomendo ...,produto
3,muito bom o produto esta tv esta entre as das...,funcionalidades
11,todos os aparelhos samsung sao de uma qualidad...,produto
12,atendeu todas as expectativas recomendo samsu...,produto


In [8]:
df_tst['labels'] =-1
df_tst.loc[df_tst["aspectsClass"] == "produto", "labels"] = 0
df_tst.loc[df_tst["aspectsClass"] == "entrega", "labels"] = 1
df_tst.loc[df_tst["aspectsClass"] == "imagem", "labels"] = 2
df_tst.loc[df_tst["aspectsClass"] == "desempenho", "labels"] = 3
df_tst.loc[df_tst["aspectsClass"] == "som", "labels"] = 4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
df = df_tst[df_tst['labels'] != -1]
df.head(10)

,text,aspectsClass,labels
0,excelente smart tv e foi entregue vem antes d...,entrega,1
1,imagem otima recursos de web bons recomendo ...,produto,0
11,todos os aparelhos samsung sao de uma qualidad...,produto,0
12,atendeu todas as expectativas recomendo samsu...,produto,0
13,rapidez boa qualidade e preocupacao em manter...,desempenho,3
14,otima imagem facil acesso a internet com wifi...,produto,0
18,produto nao deixa nada a desejar exelente go...,produto,0
28,produto excelente recomendo produto alem de ...,entrega,1
29,recomendo tem muitas funcoes praticas o wi f...,produto,0
30,maravilhosa chegou dentro do prazo estipulado ...,produto,0


In [25]:
df.labels.unique()

array([1, 0, 3, 2, 4])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["labels"], test_size = 0.3, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency
X_train_tf = tf_vectorizer.fit_transform(X_train)
X_test_tf = tf_vectorizer.transform(X_test)

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

naive_bayes_classifier = MultinomialNB()
# naive_bayes_classifier.fit(X_train_tf, y_train)
# y_pred = naive_bayes_classifier.predict(X_test_tf)

t = time()
naive_bayes_classifier.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)
print(metrics.classification_report(y_test, y_pred,zero_division = 0))
print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.002s
test time:  0.000s
accuracy:   0.795
              precision    recall  f1-score   support

           0       0.80      0.96      0.87       105
           1       0.79      0.42      0.55        36
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.79       146
   macro avg       0.40      0.34      0.35       146
weighted avg       0.77      0.79      0.76       146

confusion matrix:
[[101   4   0   0]
 [ 21  15   0   0]
 [  4   0   0   0]
 [  1   0   0   0]]
------------------------------


In [53]:
from sklearn.tree import DecisionTreeClassifier
dTree = DecisionTreeClassifier()

t = time()
dTree.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = dTree.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,zero_division = 0))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.011s
test time:  0.000s
accuracy:   0.815
              precision    recall  f1-score   support

           0       0.84      0.91      0.88       105
           1       0.77      0.64      0.70        36
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.82       146
   macro avg       0.32      0.31      0.31       146
weighted avg       0.79      0.82      0.80       146

confusion matrix:
[[96  7  1  1  0]
 [13 23  0  0  0]
 [ 0  0  0  0  0]
 [ 4  0  0  0  0]
 [ 1  0  0  0  0]]
------------------------------


In [54]:
from sklearn.svm import SVC
svm = SVC()

t = time()
svm.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = svm.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)
print(metrics.classification_report(y_test, y_pred,zero_division = 0))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.051s
test time:  0.014s
accuracy:   0.842
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       105
           1       0.85      0.61      0.71        36
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.84       146
   macro avg       0.42      0.39      0.40       146
weighted avg       0.81      0.84      0.82       146

confusion matrix:
[[101   4   0   0]
 [ 14  22   0   0]
 [  4   0   0   0]
 [  1   0   0   0]]
------------------------------


In [55]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression()

t = time()
regression.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = regression.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,zero_division = 0))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.105s
test time:  0.000s
accuracy:   0.829
              precision    recall  f1-score   support

           0       0.85      0.92      0.89       105
           1       0.77      0.67      0.72        36
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.83       146
   macro avg       0.41      0.40      0.40       146
weighted avg       0.80      0.83      0.81       146

confusion matrix:
[[97  7  1  0]
 [12 24  0  0]
 [ 4  0  0  0]
 [ 1  0  0  0]]
------------------------------


In [56]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

t = time()
mlp.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = mlp.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,zero_division = 0))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 2.920s
test time:  0.002s
accuracy:   0.842
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       105
           1       0.85      0.61      0.71        36
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.84       146
   macro avg       0.42      0.39      0.40       146
weighted avg       0.81      0.84      0.82       146

confusion matrix:
[[101   4   0   0]
 [ 14  22   0   0]
 [  4   0   0   0]
 [  1   0   0   0]]
------------------------------


In [57]:
forest = RandomForestClassifier() 

forest = forest.fit(X_train_tf, y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = forest.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,zero_division = 0))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 20.834s
test time:  0.026s
accuracy:   0.849
              precision    recall  f1-score   support

           0       0.84      0.98      0.90       105
           1       0.91      0.58      0.71        36
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1

    accuracy                           0.85       146
   macro avg       0.44      0.39      0.40       146
weighted avg       0.83      0.85      0.83       146

confusion matrix:
[[103   2   0   0]
 [ 15  21   0   0]
 [  4   0   0   0]
 [  1   0   0   0]]
------------------------------
